In [ ]:
%run setup.py build_ext --inplace

In [ ]:
%pip install xlrd
%pip install pandas

In [ ]:
from typing import NamedTuple

knot_info_database = "./KnotInfoDatabase/knotinfo_data_complete.xls"
folder_path = "./pickles/"

class KnotInput(NamedTuple):
    braid: list
    n: int  # number of punctures
    k: int  # number of E-branes

pickle = "load"
type = "_reduced"

In [ ]:
knots_braids = {
    "3_1": [1, 2, 1, 2],
    "8_2": [1, 1, 1, 1, 1, -2, 1, -2],
    "8_5": [1, 1, 1, -2, 1, 1, 1, -2],
    "8_7": [-1, -1, -1, -1, 2, -1, 2, 2],
    "8_9": [1, 1, 1, -2, 1, -2, -2, -2],
    "8_10": [-1, -1, -1, 2, -1, -1, 2, 2],
    "8_16": [-1, -1, 2, -1, -1, 2, -1, 2],
    "8_17": [1, 1, -2, 1, -2, 1, -2, -2],
    "8_18": [1, -2, 1, -2, 1, -2, 1, -2],
    "8_19": [1, 1, 1, 2, 1, 1, 1, 2],
    "8_20": [1, 1, 1, -2, -1, -1, -1, -2],
    "8_21": [1, 1, 1, 2, -1, -1, 2, 2],
}

In [ ]:
inputs = {}
for name, short_braid in knots_braids.items():
    inputs[name] = KnotInput(
        braid=[2] + [sign(s)*(abs(s)+2) for s in knots_braids[name]] + [-2],
        n=6,
        k=2,
#        k=3,
    )

# just to illustrate how to change input
#inputs["8_2"] = inputs["8_2"]._replace(max_dots=1, max_order=3)

inputs

In [ ]:
import pandas as pd
import re
from collections import defaultdict

from sage.groups.additive_abelian.additive_abelian_group import AdditiveAbelianGroup

grading_group = AdditiveAbelianGroup([0, 0])

database_answer = {}

df = pd.read_excel(knot_info_database)
for name in inputs:
    # database_answer[name][grading] has a list of invariant factors
    database_answer[name] = defaultdict(list)
#    entry_string = df[df['name'] == name]['khovanov_unreduced_integral_vector'].values[0]
    entry_string = df[df['name'] == name]['khovanov_reduced_integral_vector'].values[0]
    pattern = r'(-?\d+),\s*(-?\d+),\s*(-?\d+),\s*(-?\d+)'
    list_of_strings = re.findall(pattern, entry_string)
    for string in list_of_strings:
        khovanov_piece = tuple(int(int_str) for int_str in string)
        grading = grading_group((khovanov_piece[2], khovanov_piece[3]))
        database_answer[name][grading] += [khovanov_piece[0]]*khovanov_piece[1]
    for l in database_answer[name].values():
        l.sort()

    print(database_answer[name])

In [ ]:
from klrw.combinatorial_ebranes import CombinatorialEBrane

for name, input_data in inputs.items():
    if pickle == "load":
        print("================= Checking " + name + "=======================")
    else:
        print("==============================================================")
        print("                  Solving " + name + "                        ")
        print("==============================================================")
    E = CombinatorialEBrane(input_data.n, input_data.k)
    E.apply_braid(input_data.braid)
    try:
        hom = E.link_homology(
            ZZ,
            pickle=pickle,
            link_name=name + type,
            folder_path=folder_path,
            dualize_complex=True,
        )

        answer = defaultdict(list)
        for grading, hom_group in hom.items():
            if hom_group.order() != 1:
                answer[grading] = sorted(hom_group.invariants())

        shift = max(database_answer[name].keys()) - max(answer.keys())

        try:
            keys = frozenset(database_answer[name].keys())
            keys |= frozenset(key + shift for key in answer.keys())
            for key in sorted(keys):
                assert database_answer[name][key] == answer[key - shift]
        except AssertionError:
            print("The answer does not match!")
        else:
            print("Match!")

    except AssertionError:
        print(name + " failed.")
    except FileNotFoundError:
        print("File not found!")